In [ ]:
from jupyter_bokeh.widgets import BokehModel
from bokeh.io import output_notebook
import ipywidgets as ip
import xarray as xa
import pandas as pd
from bokeh.models import ColumnDataSource, DataTable, DateFormatter, TableColumn
from typing import List, Union, Tuple, Optional, Dict, Callable, Set

In [ ]:
class bkSpreadsheet:

    def __init__(self, data: Union[pd.DataFrame,xa.DataArray] ):
        output_notebook()
        pdf: pd.DataFrame = None
        if isinstance( data, pd.DataFrame ):
            pdf = data
        elif isinstance( data, xa.DataArray ):
            assert data.ndim == 2, f"Wrong DataArray.ndim for bkSpreadsheet ({data.ndim}): must have ndim = 2"
            pdf = data.to_pandas()
        else:
            raise TypeError( f"Unsupported data class supplied to bkSpreadsheet: {data.__class__}" )
        self._source: ColumnDataSource = ColumnDataSource( pdf )
        cids = [ str(cid) for cid in pdf.columns.values ]
        print( f"Creating table with cols = {cids}")
        self._columns = [ TableColumn(field=cid, title=cid) for cid in cids ]
        self._table = DataTable( source=self._source, columns=self._columns, width=400, height=280, selectable="checkbox" )

    def selection_callback( self, callback: Callable[[str,str,str],None] ):  # callback( attr, old, new )
        self._source.selected.on_change("indices", callback)

    def set_selection(self, indices: List[int] ):
        self._source.selected.indices = indices

    def get_selection( self ) -> List[int]:
        return self._source.selected.indices

    def gui(self) -> ip.DOMWidget:
        return BokehModel(self._table)

In [ ]:
from datetime import date
from random import randint

data = dict(
        dates=[date(2014, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )

dframe = pd.DataFrame( data )
dframe

In [ ]:
ss = bkSpreadsheet(dframe)
table_widget = ss.gui()
table_widget